# Queries with and without Azure OpenAI

Now that we have our Search Engine loaded and running, we are going to try some example queries and then use Azure OpenAI service to see if we can get even better results

## Set up variables

In [15]:
import os
import urllib
import requests
from IPython.display import display, HTML
from langchain.llms import AzureOpenAI
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain

from app.embeddings import OpenAIEmbeddings
from app.prompts import STUFF_PROMPT, REFINE_PROMPT, REFINE_QUESTION_PROMPT
from app.credentials import (
    API_VERSION,
    DATASOURCE_CONNECTION_STRING,
    AZURE_SEARCH_ENDPOINT,
    AZURE_SEARCH_KEY,
    COG_SERVICES_NAME,
    COG_SERVICES_KEY,
    AZURE_OPENAI_ENDPOINT,
    AZURE_OPENAI_KEY
)

In [16]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': AZURE_SEARCH_KEY}
params = {'api-version': API_VERSION}

## Without Azure OpenAI

In [17]:
# Index that we are going to query (from Notebook 01)
index_name = "cogsrch-index"

In [18]:
QUESTION = "Did Rosemary Papa send any emails to Ron Cozort or Kevin Cozort between June 2020 and the spring of 2022 about the CEC lawsuit?" 

# Try questions that you think might be answered or addressed in computer science papers in 2020-2021
# And compare the results with the open version of ChatGPT
# The idea is that the answers using Azure OpenAI only looks at the information contained on these publications.

# For Example:
# What is CLP?
# How Markov chains work?
# What are some examples of reinforcement learning?

In [19]:
url = AZURE_SEARCH_ENDPOINT + '/indexes/'+ index_name + '/docs'
url += '?api-version={}'.format(API_VERSION)
url += '&search={}'.format(QUESTION)
url += '&select=pages'
url += '&$top=10'
url += '&queryLanguage=en-us'
url += '&queryType=semantic'
url += '&semanticConfiguration=my-semantic-config'
url += '&$count=true'
url += '&speller=lexicon'
url += '&answers=extractive|count-3'
url += '&captions=extractive|highlight-true'
url += '&highlightPreTag=' + urllib.parse.quote('<span style="background-color: #f5e8a3">', safe='')
url += '&highlightPostTag=' + urllib.parse.quote('</span>', safe='')

resp = requests.get(url, headers=headers)
print(url)
print(resp.status_code)

search_results = resp.json()
print("Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))

https://azure-cog-search-7f52hhs73ltow.search.windows.net/indexes/cogsrch-index/docs?api-version=2021-04-30-Preview&search=Did Rosemary Papa send any emails to Ron Cozort or Kevin Cozort between June 2020 and the spring of 2022 about the CEC lawsuit?&select=pages&$top=10&queryLanguage=en-us&queryType=semantic&semanticConfiguration=my-semantic-config&$count=true&speller=lexicon&answers=extractive|count-3&captions=extractive|highlight-true&highlightPreTag=%3Cspan%20style%3D%22background-color%3A%20%23f5e8a3%22%3E&highlightPostTag=%3C%2Fspan%3E
200
Results Found: 3, Results Returned: 3


In [20]:
# Answers from semantic Search
display(HTML('<h4>Top Answers</h4>'))
for result in search_results['@search.answers']:
    if result['score'] > 0.5: # Show answers that are at least 50% of the max possible score=1
        display(HTML('<h5>' + 'Answer - score: ' + str(result['score']) + '</h5>'))
        display(HTML(result['text']))

        
# Results from key-word search
file_content = dict()

print("\n\n")
display(HTML('<h4>Top Results</h4>'))
for result in search_results['value']:
    if result['@search.rerankerScore'] > 0.4: # Show results that are at least 10% of the max possible score=4
        display(HTML('<h5>' + result['metadata_storage_name'] + '&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;score: ' + str(result['@search.rerankerScore']) + '</h5>'))
        display(HTML(result['@search.captions'][0]['text']))
        file_content[result['metadata_storage_path']]=result['pages']

## Comments on Query results

As seen above the semantic search feature of Azure Cognitive Search service is pretty good. It gives us the top answers and also the top results with the corresponding file and the paragraph where the answers is possible located
Let's see if we can make this better with Azure OpenAI

## Using Azure OpenAI

Of course we want OpenAI to give a better answer chat style, so we instead of sending this results, we send the content of this articles to OpenAI and lets GPT model give the answer.

The problem is that the content of the search result files is or can be very lengthy, more than the 4096 tokens allowed by the GPT Azure OpenAI models. So what we need to do is to split in chunks, vectorize and do a vector semantic search. 

Notice that **the documents chunks are already done in Azure Search**. file_content dictionary (created in the cell above) contains the pages (chunks) of each document. So we dont really need to chunk them again, each doc page for sure will fit on the max tokens limit of davinci 003 and embeddint-ada 002 models.


We will use a genius library call LangChain that wraps a lot of boiler plate code.

In [21]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI

os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY

In [22]:
# In Azure OpenAI create a deployment for the model "text-embedding-ada-002"
# and VERY IMPORTANT name the deployment the same: "text-embedding-ada-002"
embeddings = OpenAIEmbeddings()

In [23]:
docs = []
for key,value in file_content.items():
    for page in value:
        docs.append(Document(page_content=page, metadata={"source": key}))

In [24]:
%%time
if(len(docs)>1):
    db = FAISS.from_documents(docs, embeddings)
else:
    print("No results Found")

CPU times: user 251 ms, sys: 0 ns, total: 251 ms
Wall time: 18.3 s


In [25]:
docs_db = db.similarity_search(QUESTION)

The default prompts used by Langchain for the chain_type=refine can be found here. We modified them a bit to include language option

https://github.com/hwchase17/langchain/blob/master/langchain/chains/qa_with_sources/refine_prompts.py

In [26]:
llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0, max_tokens=500)
chain = load_qa_chain(llm, chain_type="refine", question_prompt=REFINE_QUESTION_PROMPT, refine_prompt=REFINE_PROMPT)

In [28]:
%%time

# TRY CHANGING THE LANGUAGE

response = chain({"input_documents": docs_db, "question": QUESTION, "language": "Chinese"}, return_only_outputs=True)

CPU times: user 17.8 ms, sys: 0 ns, total: 17.8 ms
Wall time: 14.6 s


In [29]:
display(HTML('<h4>Azure OpenAI Answer:</h4>'))
display(HTML(response['output_text']))

##### This answer is way better than taking just the result from Azure Cognitive Search. So the summary is:
- Azure Cognitive Search give us the top results (context)
- Azure OpenAI takes these results and understand the content and uses it as context to give the best answer
- Best of two worlds!

# Difference of ChatGPT vs GPT Smart Search results

We are using the language power of GPT-3 trained models in OpenAI to understand our questions and to respond accordingly but only within the context of our data.
Try for yourself asking the same question in ChatGPT vs here